In [1]:
# линейная регрессия
import pandas as pd                                   # для работы с датафреймом
import numpy as np                                    # для вычислений
import seaborn as sns                                 # для графиков
import matplotlib.pyplot as plt                       # для графиков
import statsmodels.api as sm                          # для статистического анализа
from scipy import stats                               # статистический анализ
from mlxtend.evaluate import permutation_test         # перестановочный тест

from sklearn.model_selection import train_test_split  # для деления на обучающую и тестовую выборки
from sklearn.impute import SimpleImputer              # для заполнения пропусков
from sklearn.preprocessing import OneHotEncoder       # для кодирования категориальных переменных
from sklearn.pipeline import Pipeline                 # для обработки данных для машинного обучения
from sklearn.preprocessing import StandardScaler      # стандартизация данных
from sklearn.compose import ColumnTransformer         # для обработки данных для машинного обучения


In [2]:
# ! pip install pingouin --upgrade

In [3]:
import pingouin as pg               # для статистического анализа

С линейной регрессией мы изучим три подхода:  

1. визуальный
2. статистический
3. машинное обучение

Важно помнить, что всегда предпочтительнее искать самый простой способ решения задачи. Если линейная регрессия будет хорошо справляться,  
то использовать более сложные способы не надо. Если эти сложные способы дадут незначительное улучшение, то не стоит их использовать, если  
это приводит к излишней трате вычислительного времени. Поэтому более сложные методы мы можем использовать, если 1) регрессия к нашей задаче  
не подходит, 2) другой метод не только лучше себя показывает, но и эффективнее.

# Загрузка данных